In [147]:
# !pip install -q rank_bm25 langchain langchain_chroma langchain_openai langchain_community langchain-text-splitters langchainhub langchain_groq

In [148]:
# pip install datasets

In [149]:
# pip install ragas

In [150]:
# pip install nest-asyncio

In [17]:
import pandas as pd
df = pd.read_csv("hf://datasets/lingjoor/databricks-dolly-15k-context-3k-rag/databricks-dolly-15k-context-32k-rag.csv")

In [18]:
len(df)

15011

In [19]:
df.head()

,instruction,context,response,category
0,When did Virgin Australia start operating?,"Virgin Australia Virgin Australia, the trading...",Virgin Australia commenced services on 31 Augu...,closed_qa
1,Which is a species of fish? Tope or Rope,Elops saurus The ladyfish or tenpounder (Elops...,Tope,classification
2,Why can camels survive for long without water?,Camel Most camels surviving today are domestic...,Camels use the fat in their humps to keep them...,open_qa
3,"Alice's parents have three daughters: Amy, Jes...",Villikins and his Dinah In Alice's Adventures ...,The name of the third daughter is Alice,open_qa
4,When was Tomoaki Komorida born?,Tomoaki Komorida Komorida was born in Kumamoto...,"Tomoaki Komorida was born on July 10,1981.",closed_qa


In [20]:
df1 = df[df['category'].isin(['closed_qa'])]
len(df1)

1773

In [45]:
df1= df1[:4]

In [46]:
import getpass
import os
import requests
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, PromptTemplate

In [49]:
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [50]:
GROQ_API_KEY= "---"

In [51]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50,is_separator_regex=False)

In [52]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [53]:
questions = df1["instruction"].values.tolist()
ground_truth = df1["response"].values.tolist()

# **Hybrid Search**

In [54]:
df1['Hybrid_rag_result'] = None
df1['Hybrid_rag_documents'] = None

<ipython-input-54-40586a766068>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Hybrid_rag_result'] = None
<ipython-input-54-40586a766068>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Hybrid_rag_documents'] = None


In [55]:
for index, row in df1.iterrows():
    # Split the context
    documents = text_splitter.create_documents([row['context']])
    document_texts = [doc.page_content for doc in documents]
    db = Chroma.from_documents(documents, OpenAIEmbeddings())
    retriever = db.as_retriever(search_kwargs={"k": 3})
    keyword_retriever = BM25Retriever.from_documents(documents)
    ensemble_retriever = EnsembleRetriever(retrievers=[retriever,keyword_retriever],weights=[0.3, 0.7])
    llm = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name="llama3-70b-8192")
    prompt = hub.pull("rlm/rag-prompt")
    hybrid_rag_chain = (
        {"context": ensemble_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    query = row['instruction']
    result = hybrid_rag_chain.invoke(query)
    df1.at[index, 'Hybrid_rag_result'] = result
    df1.at[index, 'Hybrid_rag_documents'] = document_texts

In [56]:
df1

,instruction,context,response,category,Hybrid_rag_result,Hybrid_rag_documents
0,When did Virgin Australia start operating?,"Virgin Australia Virgin Australia, the trading...",Virgin Australia commenced services on 31 Augu...,closed_qa,Virgin Australia started operating as Virgin B...,"[Virgin Australia Virgin Australia, the tradin..."
4,When was Tomoaki Komorida born?,Tomoaki Komorida Komorida was born in Kumamoto...,"Tomoaki Komorida was born on July 10,1981.",closed_qa,"Tomoaki Komorida was born on July 10, 1981.",[Tomoaki Komorida Komorida was born in Kumamot...
6,"Given a reference text about Lollapalooza, whe...",Lollapalooza Lollapalooza (Lolla) is an annual...,Lollapalooze is an annual musical festival hel...,closed_qa,Lollapalooza takes place in Grant Park in Chic...,[Lollapalooza Lollapalooza (Lolla) is an annua...
11,Who was Kyle Van Zyl playing against when he s...,Kayle van Zyl Van Zyl joined the Eastern Provi...,Kyle Van Zyl was playing against Boland U21 wh...,closed_qa,"According to the context, Kayle van Zyl scored...",[Kayle van Zyl Van Zyl joined the Eastern Prov...


In [57]:
output = df1["Hybrid_rag_result"].values.tolist()
context = df1["Hybrid_rag_documents"].values.tolist()

In [58]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : questions,
    "answer" : output,
    "contexts" : context,
    "ground_truth" : ground_truth
})

In [69]:
response_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 4
})

In [79]:
import nest_asyncio
nest_asyncio.apply()
results = evaluate(response_dataset, metrics, raise_exceptions=False)

Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]

In [84]:
df_result = results.to_pandas()

In [85]:
mean_values = df_result[['answer_relevancy', 'context_recall', 'context_precision', 'answer_correctness']].mean()

# Store the mean values in a dictionary
result = mean_values.to_dict()

# Add the 'Retriever' key
result['Retriever'] = 'Hybrid Search'

In [86]:
final_df = pd.DataFrame([result])

In [87]:
final_df

,answer_relevancy,context_recall,context_precision,answer_correctness,Retriever
0,0.714784,0.958333,0.878148,0.625853,Hybrid Search


# **CONTEXTUAL COMPRESSION RETRIEVER**

In [88]:
df1['contextual_comp_rag_result'] = None
df1['contextual_comp_rag_documents'] = None

<ipython-input-88-6643c248866b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['contextual_comp_rag_result'] = None
<ipython-input-88-6643c248866b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['contextual_comp_rag_documents'] = None


In [89]:
for index, row in df1.iterrows():
    # Split the context
    documents = text_splitter.create_documents([row['context']])
    document_texts = [doc.page_content for doc in documents]
    db = Chroma.from_documents(documents, OpenAIEmbeddings())
    retriever = db.as_retriever(search_kwargs={"k": 3})
    llm = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name="llama3-70b-8192")
    compressor = LLMChainExtractor.from_llm(llm)
    compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)
    prompt = hub.pull("rlm/rag-prompt")
    contextual_comp_rag_chain = (
        {"context": compression_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    query = row['instruction']
    result = contextual_comp_rag_chain.invoke(query)
    df1.at[index, 'contextual_comp_rag_result'] = result
    df1.at[index, 'contextual_comp_rag_documents'] = document_texts

In [91]:
output = df1["contextual_comp_rag_result"].values.tolist()
context = df1["contextual_comp_rag_documents"].values.tolist()

In [92]:
from datasets import Dataset
response_dataset = Dataset.from_dict({
    "question" : questions,
    "answer" : output,
    "contexts" : context,
    "ground_truth" : ground_truth
})

In [93]:
import nest_asyncio
nest_asyncio.apply()
results2 = evaluate(response_dataset, metrics, raise_exceptions=False)

Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]

In [97]:
df_result2 = results2.to_pandas()
mean_values = df_result2[['answer_relevancy', 'context_recall', 'context_precision', 'answer_correctness']].mean()
result = mean_values.to_dict()
result['Retriever'] = 'Contextual Compression Retriever'
result_df = pd.DataFrame([result])

In [98]:
final_df = pd.concat([final_df, result_df], ignore_index=True)

In [99]:
final_df

,answer_relevancy,context_recall,context_precision,answer_correctness,Retriever
0,0.714784,0.958333,0.878148,0.625853,Hybrid Search
1,0.250000,0.958333,0.866304,0.631374,Contextual Compression Retriever


# **RAG FUSION**

In [100]:
import langchain
from langchain.load import dumps, loads
def reciprocal_rank_fusion(results: list[list], k = 60):
  fused_scores = {}
  for docs in  results:
    for rank, doc in enumerate(docs):
      doc_str = dumps(doc)
      if doc_str not in fused_scores:
        fused_scores[doc_str] = 0
      previous_score = fused_scores[doc_str]
      fused_scores[doc_str] += 1/(rank+k)
  reranked_results = [
      (loads(doc), score)
      for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
  ]
  return reranked_results

In [101]:
df1['rag_fusion_rag_result'] = None
df1['rag_fusion_rag_documents'] = None

<ipython-input-101-fb86e42d62bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rag_fusion_rag_result'] = None
<ipython-input-101-fb86e42d62bf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rag_fusion_rag_documents'] = None


In [102]:
for index, row in df1.iterrows():
    # Split the context
    documents = text_splitter.create_documents([row['context']])
    db = Chroma.from_documents(documents, OpenAIEmbeddings())
    retriever = db.as_retriever(search_kwargs={"k": 3})
    original_question = row['instruction']
    prompt = ChatPromptTemplate(input_variables=['original_question'],
                                messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[],template='You are a helpful assistant that generates multiple search queries based on a single input query.')),
                                HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_question'], template='Generate multiple search queries related to: {question} \n OUTPUT (4 queries):'))])
    generate_queries = (
        prompt
        | ChatOpenAI(temperature=0)
        | StrOutputParser()
        | (lambda x: x.split("\n"))
    )
    retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
    final_documents = retrieval_chain_rag_fusion.invoke(original_question)
    page_contents = [doc[0].page_content for doc in final_documents]
    llm = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name="llama3-70b-8192")
    prompt2 = hub.pull("rlm/rag-prompt")
    full_rag_fusion_chain = (
        {
            "context": retrieval_chain_rag_fusion,
            "question": RunnablePassthrough()
        }
        | prompt2
        | llm
        | StrOutputParser()
    )
    result = full_rag_fusion_chain.invoke(original_question)
    df1.at[index, 'rag_fusion_rag_result'] = result
    df1.at[index, 'rag_fusion_rag_documents'] = page_contents

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [103]:
df1

,instruction,context,response,category,Hybrid_rag_result,Hybrid_rag_documents,contextual_comp_rag_result,contextual_comp_rag_documents,rag_fusion_rag_result,rag_fusion_rag_documents
0,When did Virgin Australia start operating?,"Virgin Australia Virgin Australia, the trading...",Virgin Australia commenced services on 31 Augu...,closed_qa,Virgin Australia started operating as Virgin B...,"[Virgin Australia Virgin Australia, the tradin...",Virgin Australia did not start operating on 27...,"[Virgin Australia Virgin Australia, the tradin...",Virgin Australia started operating in August 2...,[V Australia V Australia was a long-haul inter...
4,When was Tomoaki Komorida born?,Tomoaki Komorida Komorida was born in Kumamoto...,"Tomoaki Komorida was born on July 10,1981.",closed_qa,"Tomoaki Komorida was born on July 10, 1981.",[Tomoaki Komorida Komorida was born in Kumamot...,"Tomoaki Komorida was born on July 10, 1981.",[Tomoaki Komorida Komorida was born in Kumamot...,"Tomoaki Komorida was born on July 10, 1981.",[Tomoaki Komorida Komorida was born in Kumamot...
6,"Given a reference text about Lollapalooza, whe...",Lollapalooza Lollapalooza (Lolla) is an annual...,Lollapalooze is an annual musical festival hel...,closed_qa,Lollapalooza takes place in Grant Park in Chic...,[Lollapalooza Lollapalooza (Lolla) is an annua...,Lollapalooza takes place in Grant Park in Chic...,[Lollapalooza Lollapalooza (Lolla) is an annua...,Lollapalooza is an annual four-day music festi...,[Lollapalooza Lollapalooza (Lolla) is an annua...
11,Who was Kyle Van Zyl playing against when he s...,Kayle van Zyl Van Zyl joined the Eastern Provi...,Kyle Van Zyl was playing against Boland U21 wh...,closed_qa,"According to the context, Kayle van Zyl scored...",[Kayle van Zyl Van Zyl joined the Eastern Prov...,Kyle Van Zyl scored 36 of his team's points in...,[Kayle van Zyl Van Zyl joined the Eastern Prov...,Kyle Van Zyl scored 36 of his team's 61 points...,[Kayle van Zyl This included a record and a re...


In [112]:
output = df1["rag_fusion_rag_result"].values.tolist()
context = df1["rag_fusion_rag_documents"].values.tolist()

In [113]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : questions,
    "ground_truth" : ground_truth,
    "answer" : output,
    "contexts" : context,

})

In [114]:
import nest_asyncio
nest_asyncio.apply()
results3 = evaluate(response_dataset, metrics, raise_exceptions=False)

Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]

In [115]:
df_result3 = results3.to_pandas()
mean_values = df_result3[['answer_relevancy', 'context_recall', 'context_precision', 'answer_correctness']].mean()
result = mean_values.to_dict()
result['Retriever'] = 'RAG Fusion'
result_df = pd.DataFrame([result])

In [118]:
final_df = pd.concat([final_df, result_df], ignore_index=True)

In [120]:
final_df

,answer_relevancy,context_recall,context_precision,answer_correctness,Retriever
0,0.714784,0.958333,0.878148,0.625853,Hybrid Search
1,0.250000,0.958333,0.866304,0.631374,Contextual Compression Retriever
2,0.705254,0.625000,0.895833,0.726248,RAG Fusion


## **VectorStore Backed Retriever**

In [122]:
df1['vectorStore_rag_result'] = None
df1['vectorStore_rag_documents'] = None

<ipython-input-122-591f57e2529c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['vectorStore_rag_result'] = None
<ipython-input-122-591f57e2529c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['vectorStore_rag_documents'] = None


In [123]:
for index, row in df1.iterrows():
    # Split the context
    documents = text_splitter.create_documents([row['context']])
    document_texts = [doc.page_content for doc in documents]
    db = Chroma.from_documents(documents, OpenAIEmbeddings())
    retriever = db.as_retriever(search_kwargs={"k": 3})
    llm = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name="llama3-70b-8192")
    prompt = hub.pull("rlm/rag-prompt")
    hybrid_rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    query = row['instruction']
    result = hybrid_rag_chain.invoke(query)
    df1.at[index, 'vectorStore_rag_result'] = result
    df1.at[index, 'vectorStore_rag_documents'] = document_texts

In [125]:
output = df1["vectorStore_rag_result"].values.tolist()
context = df1["vectorStore_rag_documents"].values.tolist()

In [126]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : questions,
    "ground_truth" : ground_truth,
    "answer" : output,
    "contexts" : context,

})

In [131]:
# import nest_asyncio
# nest_asyncio.apply()
results4 = evaluate(response_dataset, metrics, raise_exceptions=False)

Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]

In [132]:
df_result4 = results4.to_pandas()
mean_values = df_result4[['answer_relevancy', 'context_recall', 'context_precision', 'answer_correctness']].mean()
result = mean_values.to_dict()
result['Retriever'] = 'Vector Store backed Retriever'
result_df = pd.DataFrame([result])

In [133]:
final_df = pd.concat([final_df, result_df], ignore_index=True)

In [134]:
final_df

,answer_relevancy,context_recall,context_precision,answer_correctness,Retriever
0,0.714784,0.958333,0.878148,0.625853,Hybrid Search
1,0.250000,0.958333,0.866304,0.631374,Contextual Compression Retriever
2,0.705254,0.625000,0.895833,0.726248,RAG Fusion
3,0.250000,0.958333,0.867552,0.576088,Vector Store backed Retriever


In [145]:
import plotly.graph_objs as go
colors = ['#2F4F4F', '#4682B4', '#A9A9A9', '#FFA07A']  # Dark Blue, Light Blue, Dark Gray, Light Gray
# Create traces
trace1 = go.Bar(x=final_df['Retriever'], y=final_df['answer_relevancy'], name='Answer Relevancy', marker=dict(color=colors[0]))
trace2 = go.Bar(x=final_df['Retriever'], y=final_df['context_recall'], name='Context Recall', marker=dict(color=colors[1]))
trace3 = go.Bar(x=final_df['Retriever'], y=final_df['context_precision'], name='Context Precision', marker=dict(color=colors[2]))
trace4 = go.Bar(x=final_df['Retriever'], y=final_df['answer_correctness'], name='Answer Correctness', marker=dict(color=colors[3]))
layout = go.Layout(
    title='Scores by Retriever and Metrics',
    xaxis=dict(title='Retriever', tickangle=-45),
    yaxis=dict(title='Scores', zeroline=False),
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)
fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)
fig.show()
